<a href="https://colab.research.google.com/github/anandhc6/Assignment-3/blob/main/Best_model_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Required packages

import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import backend
from random import randrange 
from tensorflow import keras
from google.colab import files
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.callbacks import EarlyStopping

In [ ]:
# Downloading dataset

!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
!tar -xf 'dakshina_dataset_v1.0.tar'

--2022-05-07 12:13:41--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.125.128, 142.250.148.128, 209.85.234.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G   212MB/s    in 9.5s    

2022-05-07 12:13:50 (201 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [ ]:
 # Paths of train and valid datasets.
train_data_path = "dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.train.tsv"
val_data_path = "dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.dev.tsv"

# Saving the files in list
with open(train_data_path, "r", encoding="utf-8") as file:
    train_data_lines = file.read().split("\n")

with open(val_data_path, "r", encoding="utf-8") as file:
    val_data_lines = file.read().split("\n")

# popping the empty character of the lists 
val_data_lines.pop()
train_data_lines.pop()

# Fixed parameter
batch_size = 64 


In [ ]:
# embedding train data

def embed_train_data(train_data_lines):

    lenk = len(train_data_lines) - 1
    train_input_data = []
    train_target_data = []
    input_data_characters = set()
    target_data_characters = set()
    
    for line in train_data_lines[: lenk]:
        target_data, input_data, _ = line.split("\t")

        # We are using "tab" as the "start sequence" and "\n" as "end sequence".
        target_data = "\t" + target_data + "\n"
        train_input_data.append(input_data)
        train_target_data.append(target_data)

        # Finding unique characters.
        for ch in input_data:
            if ch not in input_data_characters:
                input_data_characters.add(ch)
        for ch in target_data:
            if ch not in target_data_characters:
                target_data_characters.add(ch)

    print("Number of samples:", len(train_input_data))
    # adding space 
    input_data_characters.add(" ")
    target_data_characters.add(" ")

    # sorting
    input_data_characters = sorted(list(input_data_characters))
    target_data_characters = sorted(list(target_data_characters))

    # maximum length of the words
    encoder_max_length = max([len(txt) for txt in train_input_data])
    decoder_max_length = max([len(txt) for txt in train_target_data])

    print("Max sequence length for inputs:", encoder_max_length)
    print("Max sequence length for outputs:", decoder_max_length)

    # number of input and target characters
    num_encoder_tokens = len(input_data_characters)
    num_decoder_tokens = len(target_data_characters)  
    
    print("Number of unique input tokens:", num_encoder_tokens)
    print("Number of unique output tokens:", num_decoder_tokens)

    # create an index
    input_token_idx = dict([(char, i) for i, char in enumerate(input_data_characters)])
    target_token_idx = dict([(char, i) for i, char in enumerate(target_data_characters)])
   
    # creating 0 array for encoder,decoder 
    encoder_input_data = np.zeros((len(train_input_data), encoder_max_length), dtype="float32")

    decoder_input_data = np.zeros((len(train_input_data), decoder_max_length), dtype="float32")

    decoder_target_data = np.zeros((len(train_input_data), decoder_max_length, num_decoder_tokens), dtype="float32")

    # index of the character is encoded for all the sample whereas target data is one hot encoded.
    for i, (input_data, target_data) in enumerate(zip(train_input_data, train_target_data)):
        for t, char in enumerate(input_data):
            encoder_input_data[i, t] = input_token_idx[char]
        
        encoder_input_data[i, t + 1:] = input_token_idx[" "]
        
        # decoder data
        for t, char in enumerate(target_data):
            # decoder_target_data is one timestep ahead of decoder_input_data
            decoder_input_data[i, t] = target_token_idx[char]

            if t > 0:
                # excluding the start character since decoder target data is one timestep ahead.
                decoder_target_data[i, t - 1, target_token_idx[char]] = 1.0
        # append the remaining positions with empty space
       
        decoder_input_data[i, t + 1:] = target_token_idx[" "]
        decoder_target_data[i, t:, target_token_idx[" "]] = 1.0

    return encoder_input_data,decoder_input_data,decoder_target_data,num_encoder_tokens,num_decoder_tokens,input_token_idx,target_token_idx,encoder_max_length,decoder_max_length


In [ ]:
# embedding validation data

def embed_val_data(val_data_lines,num_decoder_tokens,input_token_idx,target_token_idx):
    val_input_data = []
    val_target_data = []
    lenk = len(val_data_lines) - 1

    for line in val_data_lines[: lenk]:
        target_data, input_data, _ = line.split("\t")
        
        # We use "tab" as the "start sequence" character and "\n" as "end sequence" character.
        target_data = "\t" + target_data + "\n"
        val_input_data.append(input_data)
        val_target_data.append(target_data)

    val_encoder_max_length = max([len(txt) for txt in val_input_data])
    val_decoder_max_length = max([len(txt) for txt in val_target_data])

    val_encoder_input_data = np.zeros((len(val_input_data), val_encoder_max_length), dtype="float32")
    val_decoder_input_data = np.zeros((len(val_input_data), val_decoder_max_length), dtype="float32")
    val_decoder_target_data = np.zeros((len(val_input_data), val_decoder_max_length, num_decoder_tokens), dtype="float32")

    for i, (input_data, target_data) in enumerate(zip(val_input_data, val_target_data)):
        for t, ch in enumerate(input_data):
            val_encoder_input_data[i, t] = input_token_idx[ch]
        val_encoder_input_data[i, t + 1:] = input_token_idx[" "]
        
        for t, ch in enumerate(target_data):
            # decoder_target_data is one timestep ahead of decoder_input_data
            val_decoder_input_data[i, t] = target_token_idx[ch]
            if t > 0:
                # excluding the start character since decoder target data is one timestep ahead.
                val_decoder_target_data[i, t - 1, target_token_idx[ch]] = 1.0
       
        val_decoder_input_data[i, t + 1:] = target_token_idx[" "]
        val_decoder_target_data[i, t:, target_token_idx[" "]] = 1.0

    return val_encoder_input_data,val_decoder_input_data,val_decoder_target_data,target_token_idx,val_target_data


In [ ]:
# Embedding data
encoder_input_data,decoder_input_data,decoder_target_data,num_encoder_tokens,num_decoder_tokens,input_token_idx,target_token_idx,encoder_max_length,decoder_max_length = embed_train_data(train_data_lines)

val_encoder_input_data,val_decoder_input_data,val_decoder_target_data,target_token_idx,val_target_data = embed_val_data(val_data_lines,num_decoder_tokens,input_token_idx,target_token_idx)

reverse_input_char_index = dict((i, char) for char, i in input_token_idx.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_idx.items())


Number of samples: 68217
Max sequence length for inputs: 30
Max sequence length for outputs: 28
Number of unique input tokens: 27
Number of unique output tokens: 49


In [ ]:
# Build RNN model

def seq2seq(embedding_size, n_encoder_tokens, n_decoder_tokens, n_encoder_layers,n_decoder_layers, latent_dimension, cell_type,
            target_token_idx, decoder_max_length, reverse_target_char_index,dropout,encoder_input_data, decoder_input_data,
            decoder_target_data,batch_size,epochs):
  
  encoder_inputs = keras.Input(shape=(None,), name='encoder_input')

  encoder = None
  encoder_outputs = None
  state_h = None
  state_c = None
  e_layer= n_encoder_layers
  
  # RNN

  if cell_type=="RNN":
    embed = tf.keras.layers.Embedding(input_dim=n_encoder_tokens, output_dim=embedding_size,name='encoder_embedding')(encoder_inputs)
    encoder = keras.layers.SimpleRNN(latent_dimension, return_state=True, return_sequences=True,name='encoder_hidden_1', dropout=dropout)
    print("Embed done")
    encoder_outputs, state_h = encoder(embed)
    
    for i in range(2,e_layer+1):
      layer_name = ('encoder_hidden_%d') % i
      print("Starting 2nd")
      encoder = keras.layers.SimpleRNN(latent_dimension, return_state=True, return_sequences=True,name=layer_name, dropout=dropout)
      print("Ending 2nd")

      encoder_outputs, state_h = encoder(encoder_outputs, initial_state=[state_h])

    encoder_states = None
    encoder_states = [state_h]
    decoder_inputs = keras.Input(shape=(None,), name='decoder_input')
    embed_dec = tf.keras.layers.Embedding(n_decoder_tokens, embedding_size, name='decoder_embedding')(decoder_inputs)
    
    # number of decoder layers
    d_layer = n_decoder_layers
    decoder = None
    decoder = keras.layers.SimpleRNN(latent_dimension, return_sequences=True, return_state=True,name='decoder_hidden_1', dropout=dropout)
    
    # initial state of decoder is encoder's last state of last layer
    decoder_outputs, _ = decoder(embed_dec, initial_state=encoder_states)
    for i in range(2,d_layer+1):
      layer_name = 'decoder_hidden_%d' % i
      decoder = keras.layers.SimpleRNN(latent_dimension, return_sequences=True, return_state=True,name=layer_name, dropout=dropout)
      decoder_outputs, _ = decoder(decoder_outputs, initial_state=encoder_states)
    
    decoder_dense = keras.layers.Dense(n_decoder_tokens, activation="softmax", name='decoder_output')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics=['accuracy'])                 

    model.fit(
          [encoder_input_data, decoder_input_data],
          decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
      )
    
    # Inference model
    encoder_inputs = model.input[0]
    encoder_outputs, state_h_enc = model.get_layer('encoder_hidden_' + str(n_encoder_layers)).output
    encoder_states = [state_h_enc]
    encoder_model = keras.Model(encoder_inputs, encoder_states)

    decoder_inputs = model.input[1] 
    decoder_outputs = model.get_layer('decoder_embedding')(decoder_inputs)
    decoder_states_inputs = []
    decoder_states = []

    for j in range(1, n_decoder_layers + 1):
        decoder_state_input_h = keras.Input(shape=(latent_dimension,))
        current_states_inputs = [decoder_state_input_h]
        decoder = model.get_layer('decoder_hidden_' + str(j))
        decoder_outputs, state_h_dec = decoder(decoder_outputs, initial_state=current_states_inputs)
        decoder_states += [state_h_dec]
        decoder_states_inputs += current_states_inputs

    decoder_dense = model.get_layer('decoder_output')
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    return encoder_model, decoder_model

  # GRU

  elif cell_type=="GRU":
    embed = tf.keras.layers.Embedding(input_dim=n_encoder_tokens, output_dim=embedding_size,name='encoder_embedding')(encoder_inputs)
    encoder = keras.layers.GRU(latent_dimension, return_state=True, return_sequences=True,name='encoder_hidden_1', dropout=dropout)
    encoder_outputs, state_h = encoder(embed)
    
    for i in range(2,e_layer+1):
      layer_name = ('encoder_hidden_%d') % i
      encoder = keras.layers.GRU(latent_dimension, return_state=True, return_sequences=True,name=layer_name, dropout=dropout)
      encoder_outputs, state_h = encoder(encoder_outputs, initial_state=[state_h])
    
    encoder_states = None
    encoder_states = [state_h]
    decoder_inputs = keras.Input(shape=(None,), name='decoder_input')
    embed_dec = tf.keras.layers.Embedding(n_decoder_tokens, embedding_size, name='decoder_embedding')(decoder_inputs)
    
    # number of decoder layers
    d_layer = n_decoder_layers
    decoder = None
    decoder = keras.layers.GRU(latent_dimension, return_sequences=True, return_state=True,name='decoder_hidden_1', dropout=dropout)
    
    # initial state of decoder is encoder's last state of last layer
    decoder_outputs, _ = decoder(embed_dec, initial_state=encoder_states)
    for i in range(2,d_layer+1):
      layer_name = 'decoder_hidden_%d' % i
      decoder = keras.layers.GRU(latent_dimension, return_sequences=True, return_state=True,name=layer_name, dropout=dropout)
      decoder_outputs, _ = decoder(decoder_outputs, initial_state=encoder_states)
    
    decoder_dense = keras.layers.Dense(n_decoder_tokens, activation="softmax", name='decoder_output')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

    model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics=['accuracy'])#, metrics=[my_metric]                 

    model.fit(
          [encoder_input_data, decoder_input_data],
          decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
      )
    
    # Inference Model
    encoder_inputs = model.input[0]
    encoder_outputs, state_h_enc = model.get_layer('encoder_hidden_' + str(n_encoder_layers)).output
    encoder_states = [state_h_enc]
    encoder_model = keras.Model(encoder_inputs, encoder_states)

    decoder_inputs = model.input[1]
    decoder_outputs = model.get_layer('decoder_embedding')(decoder_inputs)
    decoder_states_inputs = []
    decoder_states = []

    for j in range(1, n_decoder_layers + 1):
        decoder_state_input_h = keras.Input(shape=(latent_dimension,))
        current_states_inputs = [decoder_state_input_h]
        decoder = model.get_layer('decoder_hidden_' + str(j))
        decoder_outputs, state_h_dec = decoder(decoder_outputs, initial_state=current_states_inputs)
        decoder_states += [state_h_dec]
        decoder_states_inputs += current_states_inputs
    
    decoder_dense = model.get_layer('decoder_output')
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    return encoder_model, decoder_model

  # LSTM

  elif cell_type=="LSTM":
    embed = tf.keras.layers.Embedding(input_dim=n_encoder_tokens, output_dim=embedding_size,name='encoder_embedding')(encoder_inputs)
    encoder = keras.layers.LSTM(latent_dimension, return_state=True, return_sequences=True,name='encoder_hidden_1', dropout=dropout)
    encoder_outputs, state_h, state_c = encoder(embed)
    
    for i in range(2,e_layer+1):
      layer_name = ('encoder_hidden_%d') % i
      encoder = keras.layers.LSTM(latent_dimension, return_state=True, return_sequences=True,name=layer_name, dropout=dropout)
      encoder_outputs, state_h, state_c = encoder(encoder_outputs, initial_state=[state_h,state_c])
    
    encoder_states = None
    encoder_states = [state_h, state_c]

    decoder_inputs = keras.Input(shape=(None,), name='decoder_input')
    embed_dec = tf.keras.layers.Embedding(n_decoder_tokens, embedding_size, name='decoder_embedding')(decoder_inputs)
    
    # number of decoder layers
    d_layer = n_decoder_layers
    decoder = None
    decoder = keras.layers.LSTM(latent_dimension, return_sequences=True, return_state=True,name='decoder_hidden_1', dropout=dropout)
    
    # initial state of decoder is encoder's last state of last layer
    decoder_outputs, _,_ = decoder(embed_dec, initial_state=encoder_states)
    for i in range(2,d_layer+1):
      layer_name = 'decoder_hidden_%d' % i
      decoder = keras.layers.LSTM(latent_dimension, return_sequences=True, return_state=True,name=layer_name, dropout=dropout)
      decoder_outputs, _,_ = decoder(decoder_outputs, initial_state=encoder_states)
    
    decoder_dense = keras.layers.Dense(n_decoder_tokens, activation="softmax", name='decoder_output')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

    model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics=['accuracy'])#, metrics=[my_metric]                 
    
    model.fit(
          [encoder_input_data, decoder_input_data],
          decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
      )
    
    encoder_inputs = model.input[0]
    encoder_outputs, state_h_enc, state_c_enc = model.get_layer('encoder_hidden_' + str(n_encoder_layers)).output
    encoder_states = [state_h_enc, state_c_enc]
    encoder_model = keras.Model(encoder_inputs, encoder_states)

    decoder_inputs = model.input[1]
    decoder_outputs = model.get_layer('decoder_embedding')(decoder_inputs)
    decoder_states_inputs = []
    decoder_states = []

    for j in range(1,n_decoder_layers + 1):
        decoder_state_input_h = keras.Input(shape=(latent_dimension,))
        decoder_state_input_c = keras.Input(shape=(latent_dimension,))
        current_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        decoder = model.get_layer('decoder_hidden_' + str(j))
        decoder_outputs, state_h_dec, state_c_dec = decoder(decoder_outputs, initial_state=current_states_inputs)
        decoder_states += [state_h_dec, state_c_dec]
        decoder_states_inputs += current_states_inputs
    
    decoder_dense = model.get_layer('decoder_output')
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    return encoder_model, decoder_model
      

In [ ]:
def accuracy(val_encoder_input_data, val_target_data,n_decoder_layers,encoder_model,decoder_model, verbose=False):
        correct_count = 0
        total_count = 0
        n_val_data=len(val_encoder_input_data)
        for seq_idx in range(n_val_data):
            # Taking one sequence 
            input_charseq = val_encoder_input_data[seq_idx: seq_idx + 1]

            states_val = [encoder_model.predict(input_charseq)]*n_decoder_layers

            empty_charseq = np.zeros((1, 1))
            # adding first character of target sequence with the start character.
            empty_charseq[0, 0] = target_token_idx["\t"]
            target_charseq = empty_charseq

    
            stop_cond = False
            decoded_sentence = ""
            while not stop_cond:
                if cell_type is not None and (cell_type.lower() == 'rnn' or cell_type.lower() == 'gru'):
                    temp = decoder_model.predict([target_charseq] + [states_val])
                    output_tokens, states_val = temp[0], temp[1:]
                else:
                    temp = decoder_model.predict([target_charseq] + states_val )
                    output_tokens, states_val = temp[0], temp[1:]

                # Sample a token
                sampled_token_idx = np.argmax(output_tokens[0, -1, :])
                sampled_character = reverse_target_char_index[sampled_token_idx]
                decoded_sentence += sampled_character

                if sampled_character == "\n" or len(decoded_sentence) > decoder_max_length:
                    stop_cond = True

                # Updating the target sequence.
                target_charseq = np.zeros((1, 1))
                target_charseq[0, 0] = sampled_token_idx

            if decoded_sentence.strip() == val_target_data[seq_idx].strip():
                correct_count += 1

            total_count += 1

            if verbose:
                print('Prediction ', decoded_sentence.strip(), ',Ground Truth ', val_target_data[seq_idx].strip())
        
        accuracy =correct_count * 100.0 / total_count
        return accuracy

In [ ]:
# parameters

embedding_size=256
# n_encoder_tokens=num_encoder_tokens
# n_decoder_tokens=num_decoder_tokens
n_encoder_layers=3
n_decoder_layers=3
latent_dimension=512
cell_type='LSTM'
# target_token_idx=target_token_idx
# decoder_max_length=decoder_max_length
# reverse_target_char_index=reverse_target_char_index
dropout=0.3
epochs=2

In [ ]:
#Best model
encoder_model, decoder_model=seq2seq(embedding_size, num_encoder_tokens,num_decoder_tokens,n_encoder_layers, n_decoder_layers,latent_dimension,
                cell_type, target_token_idx, decoder_max_length,reverse_target_char_index, dropout ,encoder_input_data, decoder_input_data,
                decoder_target_data,batch_size,epochs)

# encoder_model.save('encoder_model.h5')
# decoder_model.save('decoder_model.h5')

# val_accuracy= accuracy(val_encoder_input_data, val_target_data,n_decoder_layers,encoder_model,decoder_model)
# print('Validation accuracy: ', val_accuracy)

# subset = 0
# val_accuracy = accuracy(val_encoder_input_data[0:subset], val_target_data[0:subset],n_decoder_layers,encoder_model,decoder_model) if subset>0 \
#     else accuracy(val_encoder_input_data, val_target_data,n_decoder_layers,encoder_model,decoder_model)
# print('Validation accuracy: ', val_accuracy)

Epoch 1/2
1066/1066 [==============================] - 168s 141ms/step - loss: 0.6537 - accuracy: 0.8124
Epoch 2/2
1066/1066 [==============================] - 151s 142ms/step - loss: 0.1920 - accuracy: 0.9456


In [ ]:
# compute test accuracy

print('Reading test data')
test_data_path = "dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.test.tsv"

with open(test_data_path, "r", encoding="utf-8") as f:
    test_lines = f.read().split("\n")
# popping the last element since it is empty character
test_lines.pop()

# embedding test
test_input_data = []
test_target_data = []
for line in test_lines[: (len(test_lines) - 1)]:
    target_text, input_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    test_input_data.append(input_text)
    test_target_data.append(target_text)

test_max_encoder_seq_length = max([len(txt) for txt in test_input_data])
test_max_decoder_seq_length = max([len(txt) for txt in test_target_data])
test_encoder_input_data = np.zeros((len(test_input_data), test_max_encoder_seq_length), dtype="float32")
test_decoder_input_data = np.zeros((len(test_input_data), test_max_decoder_seq_length), dtype="float32")
test_decoder_target_data = np.zeros((len(test_input_data), test_max_decoder_seq_length, num_decoder_tokens), dtype="float32")
for i, (input_text, target_text) in enumerate(zip(test_input_data, test_target_data)):
    for t, char in enumerate(input_text):
        test_encoder_input_data[i, t] = input_token_idx[char]
    test_encoder_input_data[i, t + 1:] = input_token_idx[" "]
    for t, char in enumerate(target_text):
        # decoder_target_data is one timestep ahead of decoder_input_data
        test_decoder_input_data[i, t] = target_token_idx[char]
        if t > 0:
            # excluding the start character since decoder target data is one timestep ahead.
            test_decoder_target_data[i, t - 1, target_token_idx[char]] = 1.0
    test_decoder_input_data[i, t + 1:] = target_token_idx[" "]
    test_decoder_target_data[i, t:, target_token_idx[" "]] = 1.0


Reading test data


In [ ]:
encoder_model = load_model("encoder_model.h5")
decoder_model = load_model("decoder_model.h5")

In [ ]:
def beam_search(input_seq,encoder_model,decoder_model, beam_size,n_decoder_layers,cell_type):
        temp_sequences = [([target_token_idx["\t"]], 0.0)]
        
        # Encode the input as state vectors.
        states_val = [[encoder_model.predict(input_seq)]*n_decoder_layers]

        stop_cond = False
        tt = 0
        while not stop_cond:
            char_sequences = []
            all_sequences = list()
   
            for seq, score in temp_sequences:
                char_seq = ''
                for index_a in seq:
                    char_seq += reverse_target_char_index[index_a]
                char_sequences.append((char_seq, score))
            
            tt += 1
            length_m=len(temp_sequences)
            for i in range(length_m):
                seq, score = temp_sequences[i]
                if seq[-1] == target_token_idx["\n"] or seq[-1] == target_token_idx[" "]:
                    all_sequences.append((seq, score))
                    continue
                target_sequence = np.zeros((1, 1))
                target_sequence[0, 0] = seq[-1]

                if cell_type is not None and (cell_type.lower() == 'RNN' or cell_type.lower() == 'GRU'):
                    temp = decoder_model.predict([target_sequence] + [states_val[i]])
                    output_tokens, temp_states = temp[0], temp[1:]
                else:
                    temp = decoder_model.predict([target_sequence] + states_val[i] )
                    output_tokens, temp_states = temp[0], temp[1:]

                if tt == 1:
                    states_val = [temp_states] * beam_size
                else:
                    states_val[i] = temp_states
                
                pk = len(output_tokens[0, -1, :])
                for j in range(pk):
                    candidate = (seq + [j], score - math.log(output_tokens[0, -1, j]))
                    all_sequences.append(candidate)


                sampled_token_index = np.argmax(output_tokens[0, -1, :])
                sampled_char = reverse_target_char_index[sampled_token_index]

            sorted_by_prob = sorted(all_sequences, key=lambda tup: tup[1])

            # selecting top k sequences
            temp_sequence = sorted_by_prob[:beam_size]
            if tt > decoder_max_length:
                stop_cond = True

            # all possible sequences
            char_sequences = []
            for seq, score in temp_sequences:
                char_seq = ''
                for index_d in seq:
                    char_seq += reverse_target_char_index[index_d]
                char_sequences.append((char_seq, score))

            # if every sequence has predicted \n we should stop
            all_sequences_ended = True
            for seq, _ in temp_sequences:
                if seq[-1] != target_token_idx["\n"]:
                    all_sequences_ended = False
                    break
            if all_sequences_ended:
                stop_cond = True

        # 
        char_sequences = []
        for seq, score in temp_sequences:
            char_seq = ''
            for index_c in seq:
                char_seq += reverse_target_char_index[index_c]
            char_sequences.append((char_seq, score))
        return char_sequences

In [ ]:
# Test accuracy

subset = 50
test_accuracy = accuracy(test_encoder_input_data[0:subset], test_target_data[0:subset],n_decoder_layers,encoder_model,decoder_model) if subset>0 \
    else accuracy(test_encoder_input_data, test_target_data,n_decoder_layers,encoder_model,decoder_model)
print('Validation accuracy: ', test_accuracy)

Validation accuracy:  18.0


In [ ]:
# compute test accuracy with Beam search

print('Calculating test accuracy')
test_accuracy = {}
for beamSize in range(2,3):
  df = pd.DataFrame(columns=['Source', 'Predictions', 'GroundTruth'])
  n_correct = 0
  n_total = 0
  for seq_index in range(len(test_encoder_input_data)):
      decoded_sentence = beam_search(test_encoder_input_data[seq_index:seq_index+1],encoder_model,decoder_model,beamSize,n_decoder_layers,cell_type)

      if test_target_data[seq_index].strip() == decoded_sentence[0][0].strip():
          n_correct += 1

      n_total += 1
      row = {}
      row['SourceText'] = test_input_data[seq_index].strip()
      row['GroundTruth'] = test_target_data[seq_index].strip()
      row['Prediction'] = decoded_sentence[0][0].strip()
      df = df.append(row, ignore_index=True)
  df.to_csv('predictions_'+str(beamSize)+'.csv', index=False)  
  test_accuracy[beamSize] = (n_correct * 100.0 / n_total)
print('Test accuracy ', test_accuracy)

import time
for beamSize in range(1,5):
  files.download('predictions_'+str(beamSize)+'.csv')
  time.sleep(30)

In [ ]:
def test_accuracy(val_encoder_input_data, val_target_texts,n_decoder_layers,encoder_model,decoder_model,test_input_texts ,verbose=False):
        n_correct = 0
        n_total = 0
        inputs=[]
        outputs=[]
        ground_truth=[]
        for seq_index in range(len(val_encoder_input_data)):
            # Take one sequence (part of the training set)
            # for trying out decoding.
            input_seq = val_encoder_input_data[seq_index: seq_index + 1]
            decoded_sentence = decode_sequence(input_seq,n_decoder_layers,'GRU',encoder_model,decoder_model)

            if decoded_sentence.strip() == val_target_texts[seq_index].strip():
                n_correct += 1

            n_total += 1

            if verbose:
                print('Prediction ', decoded_sentence.strip(), ',Ground Truth ', val_target_texts[seq_index].strip())
            inputs.append(test_input_texts[seq_index])
            outputs.append(decoded_sentence.strip())
            ground_truth.append(val_target_texts[seq_index].strip())
        df_train = pd.DataFrame({"Input": inputs, "Ground Truth" : ground_truth, "Model output":outputs})
        #print(df_train)
        df_train.to_csv('predictions_seq2seq2.csv', index=False)

        return n_correct * 100.0 / n_total

In [ ]:
test_accuracy = testaccuracy(test_encoder_input_data, test_target_data,n_decoder_layers,encoder_model,decoder_model)
print('Test accuracy: ', test_accuracy)